## 영화 장르 및 줄거리 기반 추천 모델 구현

### 0) 데이터셋 설명
#### tmdb_movies
- id : 각 영화에 대한 고유 ID
- title : 영화 제목
- runtime : 상영 시간
- genres : 영화 장르
- overview : 영화에 대한 간략한 설명
- popularity : TMDB에서 제공하는 인기도
- vote_avearage : TMDB에서 받은 평점 평균
- vote_count : TMDB에서 받은 투표수

#### tmdb_credits
- movie_id : 각 영화에 대한 고유 ID
- cast : 모든 출연진
- crew : 모든 제작진

In [ ]:
import pandas as pd
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
movies = pd.read_parquet('../dataset/tmdb_movies.parquet')
credits = pd.read_parquet('../dataset/tmdb_credits.parquet')

print(f"movies: {movies.shape}")
display(movies.head())

print(f"credits: {credits.shape}")
display(credits.head())

### 1) 데이터 전처리

In [ ]:
movies['overview'].iloc[2]

In [ ]:
## null값 확인
movies['overview'].isnull().sum()

In [ ]:
## 줄거리 결측치 처리 - dropna
movies.dropna(subset=['overview'], inplace=True)

movies['overview'].isnull().sum()

In [ ]:
# 'genres' 컬럼의 문자열을 list 형식으로 변환

movies['genres'] = movies['genres'].apply(eval)

In [ ]:
# 장르 추출하여 'genre_names' 컬럼에 저장

def get_genre_names(val):
        return ' '.join([i.get('name','').lower() for i in val])

movies['genres'] = movies['genres'].apply(get_genre_names)

movies.head()

In [ ]:
# overview와 genres를 합쳐서 content 컬럼 생성

movies['overview'] = movies['overview'].str.lower()
movies['content'] = movies['overview'] + ' ' + movies['genres']

movies.head()

### 2) 데이터 벡터화

In [ ]:
## TF-IDF 벡터화

tfidf = TfidfVectorizer(
    lowercase=True,
    stop_words='english'
)
tfidf_matrix = tfidf.fit_transform(movies['content'])

tfidf_matrix.shape

### 3) 유사도 계산

In [ ]:
# 코사인 유사도 계산

contents_cos_sim = cosine_similarity(tfidf_matrix)

contents_cos_sim.shape

In [ ]:
# 특정 영화의 인덱스 추출

movie_name = 'Avatar'

idx = movies[movies['title'] == movie_name].index[0]
idx

In [ ]:
# 유사도 점수 정렬

sim_scores = list(enumerate(contents_cos_sim[idx]))

sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)

sim_scores

In [ ]:
# 추천 영화 정보 출력 (자기 자신 제외)

movie_idx = [i[0] for i in sim_scores if i[0] != idx][:10]

movies.iloc[movie_idx]

### 4) 추천 함수 생성

In [ ]:
## 컨텐츠 기반 영화 추천 함수 생성

def contents_recommendation(
    dataframe: pd.DataFrame, 
    movie_name: str,
    sim_matrix,
    top_n: int = 10,
    ) -> pd.Series :

    # movie_name이 대소문자 구분 없이 필터링되도록 처리
    idx = dataframe[dataframe['title'].str.lower() == movie_name.lower()].index[0]

    sim_scores = list(enumerate(sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)

    movie_idx = [i[0] for i in sim_scores if i[0] != idx][:top_n]

    return dataframe['title'].iloc[movie_idx]

contents_recommendation(
    dataframe=movies,
    movie_name=movie_name,
    sim_matrix=contents_cos_sim,
    top_n=10
)

### 5) 유사도 행렬 저장

In [ ]:
joblib.dump(contents_cos_sim, "../models/contents_cos_sim.pkl")